## Reduced Row Echelon Form

A matrix $\mathbf{E} \in \mathcal{M}_{m \times n}(\mathbb{F})$ is said to be in **reduced row echelon form** if it satisfies the following conditions:

* if row $r$ of $\mathbf{E}$ has any nonzero entries, then the first of these is $1$;
* if $1 \leq r < s \leq m$ and rows $r$, $s$ of $\mathbf{E}$ contain nonzero entries, the first of which are $e_{rj}$ and $e_{sk}$ respectively, then $j < k$ (ie. the leading entries of a lower row occurs to the right of the leading entry of the higher row);
* if row $r$ of $\mathbf{E}$ contains nonzero entries but row $s$ does not, then $r < s$ (ie all-zero rows are at the bottom);
* if $e_{rk} = 1$ is the leading entry of row $r$ in $\mathbf{E}$, then $e_{ik} = 0$ for all $k \neq r$ (ie. a column with a leading entry contains only that leading entry). <br><br>

Given any $m \times n$ matrix $\mathbf{A}$, we can convert it to reduced row echelon form using **elementary row operations**. The three elementary row operations are:
* for any $1\leq r < s \leq m$, swap rows $r$ and $s$;
* for any $1 \leq r \leq m$ and $\lambda \neq 0$, multiply row $r$ by $\lambda$;
* for any $1 \leq r, s \leq m$ and $\lambda \in \mathbb{F}$, add $\lambda$ times row $r$ to $s$.

The following algorithm describes how a given matrix $\mathbf{A}$ can be converted to reduced row echelon form:
1. Move any all-zero rows to the bottom and swap two rows to ensure the top-left entry is non-zero.
2. Multiply row 1 by a suitable scalar to make the top-left entry equal to 1.
3. Add suitable multiples of the first row to other rows to clear the first column.
4. Repeat the above steps for each row $1 \leq r \leq m$ - finding the first non-zero entry of that row, making it equal to 1 and then clearing the column of that entry.

Here is an example. Take $$\mathbf{A} = 
\begin{pmatrix}
0 & 1 & 5 & -4 \\
1 & 4 & 3 & -2 \\
2 & 7 & 1 & -2 \\
\end{pmatrix}.$$ <br><br>

Then we have 
$$
\begin{align*}
\begin{pmatrix}
0 & 1 & 5 & -4 \\
1 & 4 & 3 & -2 \\
2 & 7 & 1 & -2 \\
\end{pmatrix} \xrightarrow {\substack{R_1 \leftrightarrow R_2 \\ R_2 \rightarrow R_2 - 2R_1}}
\begin{pmatrix}
1 & 4 & 3 & -2 \\
0 & 1 & 5 & -4 \\
0 & -1 & -5 & 2 \\
\end{pmatrix} &\xrightarrow {\substack{R_3 \rightarrow R_3 + R_2 \\ R_1 \rightarrow R_1 - 4R_2 \\ R_3 \rightarrow \frac{-1}{2} R_3}}
\begin{pmatrix}
1 & 0 & -17 & 14 \\
0 & 1 & 5 & -4 \\
0 & 0 & 0 & 1 \\
\end{pmatrix}\\
&\xrightarrow {\substack{R_2 \rightarrow R_2 + 4R_3 \\ R_1 \rightarrow R_1 - 14R_3}}
\begin{pmatrix}
1 & 0 & -17 & 0 \\
0 & 1 & 5 & 0 \\
0 & 0 & 0 & 1 \\
\end{pmatrix}.
\end{align*}
$$ <br>
We now implement this using Python.

In [2]:
import numpy as np

We will have the user input the matrix row by row. First, we ask for the number of rows (```n_rows```) and the number of columns (```n_columns```). The user must then enter the rows of the matrix, with each entry separated by a comma. <br><br> In the numpy library, a matrix may be stored using ```np.matrix```. For example, ```np.matrix('0 1 5 -4; 1 4 3 -2; 2 7 1 -2')``` stores

$$\begin{pmatrix}
0 & 1 & 5 & -4 \\
1 & 4 & 3 & -2 \\
2 & 7 & 1 & -2 \\
\end{pmatrix}.$$

We therefore have an array called `rows`, which stores the entry from each row (where each entry is separated by a space). We then apply the `row.split(" ")` to each row in `rows`. This converts each row in `rows` into an array where the the string has been broken at each space, eg. the row `'1 2 3'` becomes `['1', '2', '3']`. We also convert this to a numpy array. We then apply `astype` to convert each element to a float.

In [ ]:
n_rows = int(input("Enter the number of rows: "))
n_columns = int(input("Enter the number of columns: "))
rows = []
for i in range(n_rows):
    row = input("Enter row " + str(i+1) + ": ")
    rows.append(row.replace(',', ''))

rows = np.array([row.split(" ") for row in rows])
matrix = rows.astype(float)
matrix

We now define the three elemetary row operations:

In [ ]:
def swap_rows(matrix, r, s, factor):
    matrix[[r-1, s-1]] = matrix[[s-1, r-1]]
    factor *= -factor
    return matrix
def multiply_row(matrix, r, l, factor):
    matrix[r-1] = l * matrix[r-1]
    factor *= l
    return matrix
def add_row_multiple(matrix, l, r, s, factor):
    matrix[s-1] += l * matrix[r-1]
    return matrix

Now we must first shift all of the all-zero rows to the bottom. We therefore define the ```shift_zero_rows()``` function. <br><br>
The ```matrix.any(axis=1)``` returns an array with boolean values for each row - returning ```True``` if a row has a non-zero entry and ```False``` if the row is all-zero. We thus loop through this and store which rows are all-zero in the ```all_zero_rows``` array. We subsequently delete those rows from the matrix and append the same number of all-zero rows to the bottom of the matrix. Finally, we want the top-left entry to be equal to 1. To this end, we check if it is already 1 - in which case we simply return the matrix. Otherwise, we go through the first column until we reach the first non-zero entry and swap the first row with this row. We then divide the top-left entry by itself to ensure it is equal to 1 (which we can do since it is non-zero).

In [ ]:
def shift_zero_rows(matrix, factor):
    all_zeros = matrix.any(axis=1)
    all_zero_rows = []

    for i in range(len(all_zeros)):
        if all_zeros[i] == False:
            all_zero_rows.append(i)
    matrix = np.delete(matrix, all_zero_rows, 0)
    matrix = np.append(matrix, np.zeros( (len(all_zero_rows), n_columns) ), axis=0)
    
    if matrix[0, 0] != 0:
        matrix = multiply_row(matrix, 1, 1/matrix[0, 0], factor)
        return matrix
    else:
        for r in range(1, n_rows):
            if matrix[r, 0] == 0:
                continue
            else:
                matrix = swap_rows(matrix, 1, r+1, factor)
                matrix = multiply_row(matrix, 1, 1/matrix[0, 0], factor)
                break
        return matrix
                
            
# matrix = shift_zero_rows(matrix)

In [ ]:
matrix

Now we perform the actual row reduction. Here, **pivot** refers to the first non-zero entry of each row (where it exists) - which should be 1. <br><br> For a given row, we find the first non-zero element in that row (by iterating through the columns). We set this element to be the pivot and multiply the current row by the reciprocal of the pivot to ensure that it becomes $1$. We must now clear the entire column of the pivot. Hence, we iterate through the rows (ensuring to skip the one, on which we have the pivot). Suppose the pivot is the $(r, c)$th entry in the matrix. Then for each each $1 \leq a \leq m$ (where $m$ is the number of rows), for $a \neq r$, we apply $\rho_a := \rho_a + (-[\mathbf{A}]_{ac})\rho_r$. Repeating this for each $1 \leq r \leq m$, we complete the row reduction.

In [ ]:
def row_reduce(matrix, find_det = False):
    pivot = 0
    det = 1
    
    n_rows = matrix.shape[0]
    n_columns = matrix.shape[1]
    
    matrix = shift_zero_rows(matrix, det)
    
    for r in range(n_rows):
        for c in range(n_columns):
            if matrix[r, c] == 0:
                continue
            else:
                pivot = matrix[r, c]
                matrix = multiply_row(matrix, r+1, 1/pivot, det)
                for a in range(n_rows):
                    if a==r:
                        continue
                    else:
                        matrix = add_row_multiple(matrix, -matrix[a, c], r+1, a+1, det)
                break
        print(matrix)
        
    if find_det == False:
        return matrix
    else:
        if (matrix.shape[0] != matrix.shape[1]) == True:
            return "Matrix must be square!"
        else:
            n = matrix.shape[0]
            
        for i in range(n):
            det *= matrix[i, i]
            
        return (matrix, det)
    
    return matrix

## Inverses

We can now use this to invert matrices. We form the augmented matrix consisting of our original matrix on the left and the identity matrix on the right. It can be shown that a matrix is invertible if and only if it's reduced row echelon form is the identity. Furthermore, reducing our augmented matrix to reduced row echelon form will give us the inverse matrix on the right hand side.

In [ ]:
def invert(matrix):
    if (matrix.shape[0] != matrix.shape[1]) == True:
        return "Matrix must be square!"
    else:
        n = matrix.shape[0]

    I = np.identity(n)   
    mat = np.concatenate((matrix, I), 1)
    red = row_reduce(mat)

    if (red[:, :n]==I).all() == False:
        return "Not invertible"
    else:
        return red[:, n:]
    
    return red

We now put this all together and include determinants

In [3]:
class Matrix(object):
    def __init__(self, n_rows, n_cols, rows):
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.matrix = rows.astype(float)
        self.mat = self.matrix
        self.det = 1
    
    def swap_rows(self, r, s):
        self.matrix[[r-1, s-1]] = self.matrix[[s-1, r-1]]
        self.det = -self.det
        return self.matrix
    def multiply_row(self, r, l):
        self.matrix[r-1] = l * self.matrix[r-1]
        self.det = (1/l)*self.det
        return self.matrix
    def add_row_multiple(self, l, r, s):
        self.matrix[s-1] += l * self.matrix[r-1]
        return self.matrix
    
    def shift_zero_rows(self):
        all_zeros = self.matrix.any(axis=1)
        all_zero_rows = []

        for i in range(len(all_zeros)):
            if all_zeros[i] == False:
                all_zero_rows.append(i)
        
        self.matrix = np.delete(self.matrix, all_zero_rows, 0)
        self.matrix = np.append(self.matrix, np.zeros( (len(all_zero_rows), self.n_cols) ), axis=0)

        if self.matrix[0, 0] != 0:
            self.matrix = self.multiply_row(1, 1/self.matrix[0, 0])
            return self.matrix
        else:
            for r in range(1, self.n_rows):
                if self.matrix[r, 0] == 0:
                    continue
                else:
                    self.matrix = self.swap_rows(1, r+1)
                    self.matrix = self.multiply_row(1, 1/self.matrix[0, 0])
                    break
            return self.matrix
        
        
    def row_reduce(self, find_det = False):
        pivot = 0

        self.matrix = self.shift_zero_rows()

        for r in range(self.n_rows):
            for c in range(self.n_cols):
                if self.matrix[r, c] == 0:
                    continue
                else:
                    pivot = self.matrix[r, c]
                    self.matrix = self.multiply_row(r+1, 1/pivot)
                    for a in range(self.n_rows):
                        if a==r:
                            continue
                        else:
                            self.matrix = self.add_row_multiple(-self.matrix[a, c], r+1, a+1)
                    break
                    
            print(self.matrix)

        if find_det == False:
            return self.matrix
        else:
            if (self.n_rows != self.n_cols) == True: 
                return "Matrix must be square!"
            else:
                for i in range(self.n_rows):
                    self.det *= self.matrix[i, i]
            
                return (self.matrix, self.det)
            
    def invert(self):
        if (self.n_rows != self.n_cols) == True:
            return "Matrix must be square!"
        else:
            n = self.n_rows
            
        I = np.identity(n)   
        self.matrix = np.concatenate((self.matrix, I), 1)
        self.n_rows = self.matrix.shape[0]
        self.n_cols = self.matrix.shape[1]
        red = self.row_reduce()

        if (red[:, :n]==I).all() == False:
            return "Not invertible"
        else:
            return red[:, n:]

        return red

In [11]:
A = Matrix(2, 2, np.array(([1, 2], [3, 4])))

In [15]:
A.mat

array([[1., 2.],
       [3., 4.]])

In [13]:
A.row_reduce()

[[ 1.  2.]
 [ 0. -2.]]
[[ 1.  0.]
 [-0.  1.]]


array([[ 1.,  0.],
       [-0.,  1.]])

In [14]:
A.det

-2.0

In [16]:
A.matrix = A.mat

In [18]:
A.invert()

[[ 1.  2.  1.  0.]
 [ 0. -2. -3.  1.]]
[[ 1.   0.  -2.   1. ]
 [-0.   1.   1.5 -0.5]]


array([[-2. ,  1. ],
       [ 1.5, -0.5]])